https://towardsdatascience.com/predicting-stock-prices-using-a-keras-lstm-model-4225457f0233

https://github.com/mwitiderrick/stockprice

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# url = 'https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv'
url = 'NSE-TATAGLOBAL.csv'
dataset_train = pd.read_csv(url)
training_set = dataset_train.iloc[:, 1:2].values

In [4]:
dataset_train.head()

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55


In [8]:
# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [9]:
# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 2035):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [10]:
# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [11]:
# Part 2 - Building the RNN

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [12]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

c:\Users\vedan\Documents\Projects\PyPlotly\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [13]:
# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [14]:
# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [15]:
# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

In [16]:
# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 8s 60ms/step - loss: 0.0281
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.0040
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.0029
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 4s 62ms/step - loss: 0.0023
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - loss: 0.0030
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - loss: 0.0021
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - loss: 0.0026
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 0.0021
Epoch 9/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.0020
Epoch 10/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - loss: 0.0017
Epoch 11/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - loss: 0.0022
Epoch 12/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 6s 102ms/step - loss: 0.0021
Epoch 13/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 0.0017
Epoch 14/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - loss: 0.0016
Epoch 15/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 7s 112ms/step - 

In [19]:
# Part 3 - Making the predictions and visualising the results

# Getting the real stock price of 2017
dataset_test = pd.read_csv('tatatest.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

In [20]:
# Getting the predicted stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 76):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 554ms/step
